# [German Traffic Sign Recognition Benchmark](http://benchmark.ini.rub.de/)
### Celem jest pokazanie, jak zastosować transfer wiedzy do rozpoznawanie znaków drogowych.

### Przykłady znaków:

![](../images/gts.jpeg)

In [ ]:
import pickle

import numpy as np
np.random.seed(2019)
import pandas as pd

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout, BatchNormalization
from tensorflow.keras.utils import to_categorical

from tensorflow.keras.optimizers import Adam

from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing import image

from skimage import color, exposure
from skimage.transform import resize

from tqdm import tqdm

import matplotlib.pyplot as plt
from matplotlib.figure import Figure

from collections import Counter

%matplotlib inline

## Wczytujemy dane

In [ ]:
training_file = "../input/traffic_sign/train.p"
test_file     =  "../input/traffic_sign/test.p"

with open(training_file, mode='rb') as f:
    train = pickle.load(f)

with open(test_file, mode='rb') as f:
    test = pickle.load(f)
    
X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']

print(X_train.shape, X_test.shape)

## Przygotowanie danych

In [ ]:
if y_train.ndim == 1: y_train = to_categorical(y_train)
if y_test.ndim == 1: y_test = to_categorical(y_test)    
    
input_shape = X_train.shape[1:]
num_classes = 43

## Architektura

In [ ]:
def get_doubled_cnn3_with_dropout():
    return Sequential([
        Conv2D(32, kernel_size=(3, 3), activation='relu', padding='same', input_shape=input_shape),
        Conv2D(32, kernel_size=(3, 3), activation='relu'),
        MaxPool2D(pool_size=(2, 2)),
        Dropout(0.2),
        
        Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same'),
        Conv2D(64, kernel_size=(3, 3), activation='relu'),
        MaxPool2D(pool_size=(2, 2)),
        Dropout(0.2),
        
        Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same'),
        Conv2D(128, kernel_size=(3, 3), activation='relu'),
        MaxPool2D(pool_size=(2, 2)),
        Dropout(0.2),
        
        Flatten(), 
        
        Dense(128, activation='relu'),
        Dropout(0.2),
        Dense(num_classes, activation='softmax')
    ])


model = get_doubled_cnn3_with_dropout()
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

## Trenujemy

In [ ]:
history = model.fit(X_train, y_train,
          batch_size=128,
          epochs=3,
          verbose=1,
          validation_data=(X_test, y_test))

## Sprawdzamy

In [ ]:
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

print("CNN Error: %.2f%%" % (100-score[1]*100))

## VGG

Przygotowujemy dane:

In [ ]:
X_train_48 = []
X_test_48 = []

for i in tqdm(range(X_train.shape[0])):
    resized_img = resize(X_train[i], output_shape=(48, 48), mode='reflect')
    X_train_48.append(resized_img)
    
X_train_48 = np.array(X_train_48)
    
for i in tqdm(range(X_test.shape[0])):
    resized_img = resize(X_test[i], output_shape=(48, 48), mode='reflect')
    X_test_48.append(resized_img)
    
X_test_48 = np.array(X_test_48)

Próba #1 - zamrażamy wszystkie warstwy konwolucyjne.

In [ ]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(48, 48, 3))

for layer in base_model.layers:
    layer.trainable = False
    
base_model.summary()

In [ ]:
model = Sequential([
    base_model,
    
    Flatten(), 
        
    Dense(128, activation='relu'),
    Dropout(0.2),
    Dense(num_classes, activation='softmax')
    
])

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

In [ ]:
history = model.fit(X_train_48, y_train,
          batch_size=1024,
          epochs=3,
          verbose=1,
          validation_data=(X_test_48, y_test))

## Zadanie 7.4.1 (opcjonalnie)
Możesz spróbować jeszcze innej opcji, ale przeznacz na to maksimum 15-20 min., bo znaki drogowe są stosunkowo łatwym zadaniem. W kolejnej lekcji znajduje się trudniejsze zadanie do rozwiązania - tam bez transferu wiedzy będzie trudno osiągnąć dobry wynik.

In [ ]:
## YOUR CODE HERE

## Przydatne linki:
1. [Pipelines, Mind Maps and Convolutional Neural Networks](https://towardsdatascience.com/pipelines-mind-maps-and-convolutional-neural-networks-34bfc94db10c)
2. [ResNet for Traffic Sign Classification With PyTorch](https://towardsdatascience.com/resnet-for-traffic-sign-classification-with-pytorch-5883a97bbaa3)
3. [How to use transfer learning and fine-tuning in Keras and Tensorflow to build an image recognition system and classify (almost) any object](https://medium.com/deeplearningsandbox/how-to-use-transfer-learning-and-fine-tuning-in-keras-and-tensorflow-to-build-an-image-recognition-94b0b02444f2)